In [1]:
import gym
from IPython.display import clear_output
import numpy as np
import time
from pprint import pprint

In [2]:
env = gym.make('FrozenLake8x8-v0')

In [3]:
gamma = 0.995

In [4]:
# Define State Action Space
state_space_n = env.observation_space.n
action_space_n = env.action_space.n
state_action_space_n = state_space_n * action_space_n
# Map State-Action
def msa(state, action):
    return state * action_space_n + action
# Build Reward & Transition Matrix
R = np.zeros([state_action_space_n])
P = np.zeros([state_action_space_n, state_space_n])
for state in range(state_space_n):
    for action in range(action_space_n):
        for p, s, r, _ in env.P[state][action]:
            R[msa(state, action)] += p * r
            P[msa(state, action), s] = p

In [5]:
%%time
# Calculate Optimal Q Values
Q = np.zeros([state_action_space_n])
PI = np.zeros([state_space_n, state_action_space_n])
I = np.identity(state_action_space_n)
for i in range(100):
    # print(i)
    # Build Policy Matrix
    old_PI = PI.copy()
    PI.fill(0)
    for state in range(state_space_n):
        PI[state, msa(state,0) + np.argmax(Q[msa(state,0):msa(state,action_space_n)])] = 1
    # print('PI', np.array_equal(old_PI, PI), state_space_n - sum(np.sum(old_PI == PI, axis=1) == state_action_space_n))
    # Calculate Q Values
    old_Q = Q
    Q = np.linalg.solve(I - gamma * P @ PI, R)
    # Q = np.linalg.inv(I - gamma * P @ PI) @ R
    # print('Q ', np.array_equal(Q, old_Q), state_action_space_n - sum(Q == old_Q))
    if np.array_equal(Q, old_Q): break

Wall time: 627 ms


In [6]:
total = 0
for _ in range(5000):
    state = env.reset()
    i = 0
    while True:
        i += 1
        action = np.argmax(Q[msa(state,0):msa(state,action_space_n)])
        state, reward, done, info = env.step(action)

        # clear_output(wait=True)
        # env.render()
        # print(i)
        # time.sleep(0.01)
        if done:
            if not reward: i = 200
            break
    total += i
print(total / 5000)
env.close()

102.2986
